# Demo to play around with langchain summarization


In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/huggingface/hub'
# os.environ['HUGGINGFACE_HUB_CACHE'] = '/mnt/huggingface/hub/'

import langchain
import torch

import transformers
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

/data/anaconda/envs/logzilla/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load model

In [3]:
model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name
)

config.json: 100%|██████████| 571/571 [00:00<00:00, 3.97MB/s]


In [26]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

config.json: 100%|██████████| 571/571 [00:00<00:00, 4.97MB/s]
pytorch_model.bin.index.json: 100%|██████████| 23.9k/23.9k [00:00<00:00, 99.2MB/s]
pytorch_model-00001-of-00002.bin:  53%|█████▎    | 5.30G/9.94G [00:24<00:21, 212MB/s]


KeyboardInterrupt: 

Tokenize text

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Bitsandbytes

In [10]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


/data/anaconda/envs/logzilla/lib/python3.11/site-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 9943.03 MB. The target location /home/hackathon14/.cache/huggingface/hub only has 0.00 MB free disk space.
  warnings.warn(
/data/anaconda/envs/logzilla/lib/python3.11/site-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 9943.03 MB. The target location /home/hackathon14/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/blobs only has 0.00 MB free disk space.
  warnings.warn(
pytorch_model-00001-of-00002.bin:   0%|          | 10.5M/9.94G [00:00<00:52, 190MB/s]


OSError: [Errno 28] No space left on device